In [ ]:
# default_exp core

# core

> Core utilities for working with DICOM files. Utilizes the `pydicom` and `fastcore` packages. Some ideas borrowed from [fastai.medical.image](https://github.com/fastai/fastai/blob/master/fastai/medical/imaging.py).

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export
from dicomtools.basics import *

import pydicom
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue

In [4]:
#export
@patch
def dcmread(fn: Path, no_pixels=True, force=True):
    "Reads a DICOM file and returns the corresponding pydicom.Dataset"
    return pydicom.dcmread(str(fn), stop_before_pixels=no_pixels, force=force)

In [5]:
#export
def _cast_dicom_special(x):
    cls = type(x)
    if not cls.__module__.startswith('pydicom'): return x
    if cls.__base__ == object: return x
    return cls.__base__(x)

def _split_elem(res, k, v):
    if not isinstance(v, DcmMultiValue): return
    for i, o in enumerate(v): res[f'{k}{"" if i == 0 else i}'] = o

In [6]:
#export
_cols = [
    'SeriesDescription', # needed for labeling series
    'SequenceName', # may be used for labeling series
    'StudyDescription', # to filter on "MRI BRAIN WITH AND WITHOUT CONTRAST" in some cases
    'SOPClassUID', # to filter on "MR Image Storage"
    'BodyPartExamined', # to filter on "HEAD" or "BRAIN"
    'ImageOrientationPatient', # to calculate slice orientation (e.g. axial, coronal, sagittal)
    'EchoTime',
    'InversionTime',
    'EchoTrainLength',
    'RepetitionTime',
    'TriggerTime',
    'SequenceVariant',
    'ScanOptions',
    'ScanningSequence',
    'MRAcquisitionType',
    'ImageType',
    'PixelSpacing',
    'SliceThickness',
    'PhotometricInterpretation',
    'ContrastBolusAgent',
    'AngioFlag', # addition to list from paper
    'DiffusionBValue' # addition to list from paper
]

In [7]:
#export
@patch
def as_dict(self: DcmDataset, filt=True, split_multi=False):
    if filt:
        vals = [self[o] for o in self.keys() if self[o].keyword in _cols]
    else:
        vals = [self[o] for o in self.keys()]
    items = [(v.keyword, v.value.name) if v.keyword == 'SOPClassUID' else (v.keyword, v.value) for v in vals]
    res = dict(items)
    res['fname'] = self.filename
    if split_multi:
        for k, v in items: _split_elem(res, k, v)
        for k in res: res[k] = _cast_dicom_special(res[k])
    return res

In [8]:
#export
def _dcm2dict(fn, excl_private=False, **kwargs): 
    ds = fn.dcmread(**kwargs)
    if excl_private: ds.remove_private_tags()
    return ds.as_dict(**kwargs)

In [9]:
#export
@delegates(parallel)
def _from_dicoms(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(_dcm2dict, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms = classmethod(_from_dicoms)